In [1]:
import pandas as pd
import PyPDF2 as Pypdf
import re

In [2]:
def getPDFContent(path, first_page, last_page):
    content = []
    pdf = Pypdf.PdfFileReader(path)
    for i in range(first_page, last_page):
        content += pdf.getPage(i).extractText().splitlines()
    return content

In [3]:
text = getPDFContent('2016ChicagoCouncilSurvey-FinalQuestionnaire.pdf', 0, 19)
text

[' ',
 ' ',
 ' ',
 '310.209.00945.1_Chicago Council_2016 Biannual Survey_Main_Final Questionnaire_071216',
 ' ',
 ' ',
 '1',
 '/',
 '19',
 ' ',
 'The Chicago Council on Global Affairs',
 ' ',
 '',
 ' ',
 '2016 ',
 'Biannual',
 ' ',
 'Survey',
 ' ',
 'May ',
 '2016',
 ' ',
 '310.209.00945.1',
 ' ',
 'Maine ',
 'Questionnaire',
 ' ',
 'Account Executive: ',
 'Chris Moesnner',
 ' ',
 'Project Manager: ',
 'McPetrie Linda',
 ' ',
 'Contracted Length of Interview: ',
 '2',
 '5',
 ' ',
 'minutes',
 ' ',
 'Start Fieldwork: ',
 '06/',
 '10/16',
 ' ',
 'End Fieldwork: ',
 '06/',
 '24/16',
 ' ',
 'Document Name: ',
 '310.209.00945.1_ 2016 public opinion survey_Pretest_V0.01',
 ' ',
 ' ',
 ' ',
 'I. SAMPLE VARIABLES',
 ' ',
 ' ',
 'N/A',
 ' ',
 ' ',
 ' ',
 'II. QUOTA CHECK BASED ON SAMPLE VARIABLES',
 ' ',
 'N/A',
 ' ',
 ' ',
 ' ',
 'III. ',
 'INTRODUCTION',
 ' ',
 ' ',
 ' ',
 '',
 'the big',
 ' ',
 'foreign policy issues',
 ' ',
 'of the day',
 '.  There is no right ',
 'or wrong answer and plea

In [93]:
def is_valid(txt):
    pattern1_invalid = re.compile('(Q[0-9]+_*[0-9A-Za-z]*)')
    pattern2_invalid = re.compile('Base\: ')
    pattern3_invalid = re.compile('Answers in column[.]*')
    if pattern1_invalid.match(txt):
        return False
    elif pattern2_invalid.match(txt):
        return False
    elif pattern3_invalid.match(txt):
        return False
    else:
        return True

In [156]:
def find_questions(text):

    questions_found = set()
    question_tuples = []
    for i in range(len(text)):
        pattern1 = re.findall('(Q[0-9]+_*[0-9A-Za-z]*)(.*)', text[i])
        if pattern1:
            if pattern1[0][0] not in questions_found:
                question = pattern1[0][1]
                for j in range(1,1000):
                    if i+j < len(text):
                        pattern2 = re.findall('(Q[0-9]+_*[0-9A-Za-z]*)(.*)', text[i+j])
                        if pattern2:
                            if pattern2[0][0] == pattern1[0][0]:
                                question += text[i+j]
                        elif is_valid(text[i+j]):
                            if text[i+j]:
                                question += text[i+j]
                        else:
                            break
                    else:
                        break
                num = pattern1[0][0]
                questions_found.add(num)
                pattern_cleaned = (num, question)
                question_tuples.append(pattern_cleaned)
    return question_tuples

In [181]:
question_tuples = find_questions(text)

In [182]:
question_tuples

[('Q2',
  "/4. News about the relations of the United States with other countries   Answers in column:  1. Very interested 2. Somewhat interested 3. Hardly interested 4. Don't follow the news   "),
 ('Q3_1',
  '. [S] In order to make sure that your browser is working correctly, please select number 4 from the below list.  1. 1 2. 2 3. 3  4. 4 5. 5   '),
 ('Q3',
  '. [S] Do you think it will be best for the future of the country if we take an active part in world affairs or if we stay out of world affairs?  1. Active part 2. Stay out   Scripter: Please create DOV_TRACKAB and randomly assign and record respondents to a value of 1   '),
 ('Q5_NEW',
  '. [Grid, SP Across MP Down] Below is a list of possible threats to the vital interest of the United States in the next 10 years. For each one, please select whether you see this as a critical threat, an important but not critical threat, or not an important threat at all:  Scripter: Please randomly select and record 10 statements to show the

In [183]:
question_tuples = question_tuples[2:-7]

In [184]:
question_tuples

[('Q3',
  '. [S] Do you think it will be best for the future of the country if we take an active part in world affairs or if we stay out of world affairs?  1. Active part 2. Stay out   Scripter: Please create DOV_TRACKAB and randomly assign and record respondents to a value of 1   '),
 ('Q5_NEW',
  '. [Grid, SP Across MP Down] Below is a list of possible threats to the vital interest of the United States in the next 10 years. For each one, please select whether you see this as a critical threat, an important but not critical threat, or not an important threat at all:  Scripter: Please randomly select and record 10 statements to show the respondents.  Of the 10 statements selected, please randomize and record the order of the 10 statements shown.   Statements in row:  5/1. Drug related violence and instability in Mexico 5/4. Islamic fundamentalism 5/5. The possibility of unfriendly countries becoming nuclear powers 5/6a. Large numbers of immigrants and refugees coming into the U.S. 5/8a

In [187]:
question_tuples = question_tuples[0:26] + question_tuples[29:]

In [188]:
question_tuples

[('Q3',
  '. [S] Do you think it will be best for the future of the country if we take an active part in world affairs or if we stay out of world affairs?  1. Active part 2. Stay out   Scripter: Please create DOV_TRACKAB and randomly assign and record respondents to a value of 1   '),
 ('Q5_NEW',
  '. [Grid, SP Across MP Down] Below is a list of possible threats to the vital interest of the United States in the next 10 years. For each one, please select whether you see this as a critical threat, an important but not critical threat, or not an important threat at all:  Scripter: Please randomly select and record 10 statements to show the respondents.  Of the 10 statements selected, please randomize and record the order of the 10 statements shown.   Statements in row:  5/1. Drug related violence and instability in Mexico 5/4. Islamic fundamentalism 5/5. The possibility of unfriendly countries becoming nuclear powers 5/6a. Large numbers of immigrants and refugees coming into the U.S. 5/8a

In [189]:
def to_csv(tuple_list, name):
    cleaned_data = pd.DataFrame()
    for tup in question_tuples:
        var_name = tup[0]
        question = tup[1]
        cleaned_data = cleaned_data.append([[var_name, question]], ignore_index=True)
    cleaned_data.columns = ["Var_Name", "Var_Text"]
    cleaned_data.to_csv(path_or_buf=name + ".csv", index=False)

In [190]:
to_csv(question_tuples, "2016CCSSurvey")